# Clasificación de Imágenes de Autos con CNN (CIFAR-10)
**Autor: Juan Ángel Vizcarra Aldana**

Este notebook entrena un modelo de red neuronal (CNN) usando el dataset CIFAR-10 para identificar imágenes de automóviles (clase 1) contra otras clases (clase 0).

### ¿Qué hace este notebook?
- Carga y normaliza el dataset CIFAR-10.
- Convierte la tarea en un problema de clasificación binaria (Auto vs No Auto).
- Entrena un modelo de red neuronal.
- Muestra métricas y gráficas de desempeño.

> 💡 Ejecuta las celdas en orden para asegurar el correcto funcionamiento.


# Clasificación de Imágenes de Autos con CNN (CIFAR-10)
**Autor: Juan Ángel Vizcarra Aldana**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train_bin = (y_train.flatten() == 1).astype(int)
y_test_bin = (y_test.flatten() == 1).astype(int)
y_train_cat = to_categorical(y_train_bin, 2)
y_test_cat = to_categorical(y_test_bin, 2)

In [ ]:
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])
cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

In [ ]:
history_cnn = cnn_model.fit(x_train, y_train_cat, epochs=10, batch_size=64, 
                          validation_split=0.2, verbose=1)

In [ ]:
loss, acc = cnn_model.evaluate(x_test, y_test_cat, verbose=0)
print(f"Precisión en test (CNN): {acc:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_cnn.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history_cnn.history['val_loss'], label='Pérdida Validación')
plt.title('CNN - Curva de Pérdida')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_cnn.history['accuracy'], label='Precisión Entrenamiento')
plt.plot(history_cnn.history['val_accuracy'], label='Precisión Validación')
plt.title('CNN - Curva de Precisión')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
y_pred_cnn = cnn_model.predict(x_test).argmax(axis=1)
disp = ConfusionMatrixDisplay.from_predictions(y_test_bin, y_pred_cnn, display_labels=['No Auto', 'Auto'])
disp.ax_.set_title('Matriz de Confusión - CNN')
plt.show()

In [ ]:
# Mostrar 10 imágenes del conjunto de test: 5 autos y 5 no-autos
import matplotlib.pyplot as plt

clases = {1: "Auto", 0: "No Auto"}  # 1 = coche en CIFAR-10

fig, axs = plt.subplots(2, 5, figsize=(12, 6))
autos_mostrados = 0
no_autos_mostrados = 0
i = 0

while autos_mostrados < 5 or no_autos_mostrados < 5:
    img = x_test[i]
    etiqueta = y_test_bin[i]

    if etiqueta == 1 and autos_mostrados < 5:
        axs[0, autos_mostrados].imshow(img)
        axs[0, autos_mostrados].set_title(f"{clases[1]}")
        axs[0, autos_mostrados].axis('off')
        autos_mostrados += 1
    elif etiqueta == 0 and no_autos_mostrados < 5:
        axs[1, no_autos_mostrados].imshow(img)
        axs[1, no_autos_mostrados].set_title(f"{clases[0]}")
        axs[1, no_autos_mostrados].axis('off')
        no_autos_mostrados += 1

    i += 1

plt.suptitle("Ejemplos CIFAR-10: Autos vs No-Autos")
plt.tight_layout()
plt.show()
